In [1]:
from typing import Tuple

import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'japan'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

model = Prophet().fit(df_train)

03:01:10 - cmdstanpy - INFO - Chain [1] start processing
03:01:10 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,2.388686,1.969981,2.823820,2.388686,2.388686,-0.000715,-0.000715,-0.000715,-0.000715,-0.000715,-0.000715,0.0,0.0,0.0,2.387971
1,2022-04-01,2.375351,1.939465,2.779279,2.375351,2.375351,-0.003765,-0.003765,-0.003765,-0.003765,-0.003765,-0.003765,0.0,0.0,0.0,2.371585
2,2022-05-01,2.362446,1.957817,2.753710,2.362446,2.362446,-0.006281,-0.006281,-0.006281,-0.006281,-0.006281,-0.006281,0.0,0.0,0.0,2.356165
3,2022-06-01,2.349110,1.947411,2.802857,2.349110,2.349110,0.013593,0.013593,0.013593,0.013593,0.013593,0.013593,0.0,0.0,0.0,2.362704
4,2022-07-01,2.336205,1.923448,2.742669,2.336205,2.336205,-0.004793,-0.004793,-0.004793,-0.004793,-0.004793,-0.004793,0.0,0.0,0.0,2.331412
5,2022-08-01,2.322870,1.921160,2.738515,2.322870,2.322870,0.009289,0.009289,0.009289,0.009289,0.009289,0.009289,0.0,0.0,0.0,2.332159
6,2022-09-01,2.309535,1.853586,2.731570,2.309048,2.309689,-0.003220,-0.003220,-0.003220,-0.003220,-0.003220,-0.003220,0.0,0.0,0.0,2.306315
7,2022-10-01,2.296630,1.886876,2.715825,2.294130,2.297416,-0.002460,-0.002460,-0.002460,-0.002460,-0.002460,-0.002460,0.0,0.0,0.0,2.294170
8,2022-11-01,2.283295,1.862051,2.711933,2.279861,2.284995,-0.009179,-0.009179,-0.009179,-0.009179,-0.009179,-0.009179,0.0,0.0,0.0,2.274116
9,2022-12-01,2.270390,1.843896,2.680121,2.265155,2.273832,-0.012374,-0.012374,-0.012374,-0.012374,-0.012374,-0.012374,0.0,0.0,0.0,2.258016


In [11]:
rmse

0.24962579469274235

In [12]:
with open(f'{country}_prophet_base_model.json', 'w') as f:
    f.write(model_to_json(model))

In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [14]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,2.231675,1.826637,2.648746,2.231675,2.231675,0.006813,0.006813,0.006813,0.006813,0.006813,0.006813,0.0,0.0,0.0,2.238487
1,2023-04-01,2.218339,1.782431,2.638156,2.218339,2.218339,-0.006101,-0.006101,-0.006101,-0.006101,-0.006101,-0.006101,0.0,0.0,0.0,2.212238
2,2023-05-01,2.205434,1.766681,2.626681,2.205434,2.205434,-0.019006,-0.019006,-0.019006,-0.019006,-0.019006,-0.019006,0.0,0.0,0.0,2.186428
